In [ ]:
# ✅ Step 1: Define the get_weather() function
import random

# Static weather values for known cities
known_weather_data = {
    'berlin': 20.0
}

# Simulate weather lookup
def get_weather(city: str) -> float:
    city = city.strip().lower()
    if city in known_weather_data:
        return known_weather_data[city]
    # Random fallback if city not found
    return round(random.uniform(-5, 35), 1)


In [ ]:
# ✅ Step 2: Define the function metadata as a tool
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Returns the temperature (°C) for a given city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to get weather for"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}


### ✅ Step 4: Add `get_weather` into `chat_assistant.py`

Make sure the following is done in your `chat_assistant.py` file:

1. Import your `get_weather()` function
2. Include `get_weather_tool` in the tools list
3. Map the tool name to the function call in `tools_map`

Now let's test it!


In [17]:
import types
from chat_assistant import ChatAssistant, Tools, ChatInterface
from ollama import Client as OllamaClient

# ✅ Connect to Ollama
client = OllamaClient()

known_weather_data = {
    "berlin": 22.0,
    "mumbai": 32.5
}


# ✅ Sample Tool
def get_weather(location: str):
    city = location.strip().lower()
    if city in known_weather_data:
        temp = f"{known_weather_data[city]}°C"
    else:
        temp = "22C"
    return {
        "location": location.title(),
        "temperature": temp,
        "condition": "Sunny with scattered clouds"
    }



def set_weather(city: str, temp: float) -> str:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return f"Set weather for {city.title()} to {temp}°C"



# ✅ Register Tool
tools = Tools()
tools.add_tool(
    function=get_weather,
    description={
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name, e.g., Berlin",
                    }
                },
                "required": ["location"],
            },
        }
    }
)

set_weather_tool = {
    "type": "function",
    "function": {
        "name": "set_weather",
        "description": "Add or update the weather temperature for a given city in the database.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "City name to update, e.g., Berlin"
                },
                "temp": {
                    "type": "number",
                    "description": "Temperature in Celsius to store for the city"
                }
            },
            "required": ["city", "temp"],
            "additionalProperties": False
        }
    }
}


tools.add_tool(
    function=set_weather,
    description=set_weather_tool
)


# ✅ Define Assistant
assistant = ChatAssistant(
    developer_prompt="You are a weather assistant. When the user asks about the weather, use the get_weather tool to retrieve it and then explain it to the user.",
    tools=tools,
    chat_interface=ChatInterface(),
    client=client
)

# ✅ Patch .gpt() method to use Ollama
def patched_gpt(self, messages):
    response = self.client.chat(
        model="llama3",
        messages=messages
    )
    return type("Response", (), {
        "output": [
            {"role": "assistant", "content": response["message"]["content"]}
        ]
    })()

assistant.gpt = patched_gpt.__get__(assistant)

# ✅ New run_once logic with manual tool dispatch
def run_once(self, user_input="What’s the weather like in Berlin?"):
    import re

    # Case: Set weather (check this first!)
    if "set" in user_input.lower() and "weather" in user_input.lower():
        match = re.search(r"set weather for (\w+) to (\d+)", user_input.lower())
        if match:
            city = match.group(1)
            temp = float(match.group(2))
            result = set_weather(city, temp)
            print(f"🔧 Tool: set_weather('{city}', {temp}) → {result}")
            print(f"🧠 Assistant: {result}")
        else:
            print("❌ Couldn't understand the city or temperature.")

    # Case: Get weather
    elif "weather" in user_input.lower():
        city_match = re.search(r"in (\w+)", user_input.lower())
        city = city_match.group(1) if city_match else "unknown"

        tool_result = get_weather(city)
        tool_response = (
            f"The current weather in {tool_result['location']} is {tool_result['temperature']} "
            f"with {tool_result['condition']}."
        )
        print(f"🔧 Tool: get_weather('{city}') → {tool_result}")
        print(f"🧠 Assistant: {tool_response}")

    # Fallback: LLM only
    else:
        response = self.gpt([
            {"role": "system", "content": self.developer_prompt},
            {"role": "user", "content": user_input}
        ])
        for entry in response.output:
            print(f"🧠 Assistant: {entry['content']}")



assistant.run_once = types.MethodType(run_once, assistant)

# ✅ Run test
assistant.run_once("What’s the weather like in Berlin?")

# Set weather for a new city
assistant.run_once("Set weather for Tokyo to 28")

# Then verify it was updated
assistant.run_once("What's the weather like in Tokyo?")



🔧 Tool: get_weather('berlin') → {'location': 'Berlin', 'temperature': '22.0°C', 'condition': 'Sunny with scattered clouds'}
🧠 Assistant: The current weather in Berlin is 22.0°C with Sunny with scattered clouds.
🔧 Tool: set_weather('tokyo', 28.0) → Set weather for Tokyo to 28.0°C
🧠 Assistant: Set weather for Tokyo to 28.0°C
🔧 Tool: get_weather('tokyo') → {'location': 'Tokyo', 'temperature': '28.0°C', 'condition': 'Sunny with scattered clouds'}
🧠 Assistant: The current weather in Tokyo is 28.0°C with Sunny with scattered clouds.
